## Ignis
Ignis framework that contains various functionalities such as characterization, verification and mitigation.

### Generating Noise effects of relaxation

noise type as decoherence.
3 main types of decoherence are
- $T_1$ relaxation
- $T_2/T_2^*$ dephasing

### Estimating $T_1$ decoherence time

### Generating the noise effects of dephasing

### Estimating $T_2$ decoherence time

### Estimating $T_2$ * ($T_2$ star) decoherence time

### Visualizing the $T_1$, $T_2$, and $T_2^*$ decoherence times

### Mitigating readout errors using measurement calibrations